In [1]:
# Python 함수를 Component 로 바꿔주는 함수
# decorator 로도 사용할 수 있으며, 여러 옵션을 argument 로 설정할 수 있음
# add_op = create_component_from_func(
#                 func=add,
#                 base_image='python:3.7', # Optional : component 는 k8s pod 로 생성되며, 해당 pod 의 image 를 설정
#                 output_component_file='add.component.yaml', # Optional : component 도 yaml 로 compile 하여 재사용하기 쉽게 관리 가능
#                 packages_to_install=['pandas==0.24'], # Optional : base image 에는 없지만, python code 의 의존성 패키지가 있으면 component 생성 시 추가 가능
#             )
# from kfp.components import create_component_from_func

from kfp import dsl

"""
kfp.components.create_component_from_func :
    Python 함수를 Component 로 바꿔주는 함수
    decorator 로도 사용할 수 있으며, 여러 옵션을 argument 로 설정할 수 있음
    
    add_op = create_component_from_func(
                func=add,
                base_image='python:3.7', # Optional : component 는 k8s pod 로 생성되며, 해당 pod 의 image 를 설정
                output_component_file='add.component.yaml', # Optional : component 도 yaml 로 compile 하여 재사용하기 쉽게 관리 가능
                packages_to_install=['pandas==0.24'], # Optional : base image 에는 없지만, python code 의 의존성 패키지가 있으면 component 생성 시 추가 가능
            )
"""


def add(value_1: int, value_2: int) -> int:
    """
    더하기
    """
    ret = value_1 + value_2
    return ret


def subtract(value_1: int, value_2: int) -> int:
    """
    빼기
    """
    ret = value_1 - value_2
    return ret


def multiply(value_1: int, value_2: int) -> int:
    """
    곱하기
    """
    ret = value_1 * value_2
    return ret


# Python 함수를 선언한 후, kfp.components.create_component_from_func 를 사용하여
# ContainerOp 타입(component)으로 convert
# add_op = create_component_from_func(add)
# subtract_op = create_component_from_func(subtract)
# multiply_op = create_component_from_func(multiply)
add_op = dsl.component(add)
subtract_op = dsl.component(subtract)
multiply_op = dsl.component(multiply)

from kfp.dsl import pipeline


@pipeline(name="add example")
def my_pipeline(value_1: int, value_2: int):
    task_1 = add_op(value_1=value_1, value_2=value_2)
    task_2 = subtract_op(value_1=value_1, value_2=value_2)

    # component 간의 data 를 넘기고 싶다면,
    # output -> input 으로 연결하면 DAG 상에서 연결됨

    # compile 된 pipeline.yaml 의 dag 파트의 dependency 부분 확인
    # uploaded pipeline 의 그래프 확인
    task_3 = multiply_op(value_1=task_1.output, value_2=task_2.output)

/opt/conda/lib/python3.11/site-packages/kfp/dsl/component_decorator.py:119: FutureWarning: Python 3.7 has reached end-of-life. The default base_image used by the @dsl.component decorator will switch from 'python:3.7' to 'python:3.8' on April 23, 2024. To ensure your existing components work with versions of the KFP SDK released after that date, you should provide an explicit base_image argument and ensure your component works as intended on Python 3.8.
  return component_factory.create_component_from_func(


In [4]:
import kfp

# kfp dsl compile --py test.py --output pipeline_test.yaml
kfp.compiler.Compiler().compile(my_pipeline, "pipeline_test.yaml")

In [31]:
import requests

response = requests.post(
    "https://keycloak.clownchrys.link/realms/default/protocol/openid-connect/token",
    data={
        "client_id": "kubeflow-oidc-authservice",
        "client_secret": "SM7BlsEB0DZW6noQsgcZRw==",
        "username": "admin",
        "password": "thisisauthorized",
        "grant_type": "password"
    }
).json()

In [32]:
client = kfp.Client(
    host="http://ml-pipeline.kubeflow.svc.cluster.local:8888",
    verify_ssl=False,
    existing_token=response["access_token"]
)
client

In [34]:
run = client.create_run_from_pipeline_func(my_pipeline, arguments={"value_1": 1, "value_2": 2})
run

RunPipelineResult(run_id=48a94337-05d7-46cc-8bd4-965ac08def07)